## Install libraries

In [1]:
%%capture
!pip install emoji regex pandas unicodedata fasttext

In [2]:
import pandas as pd
import re
import regex
import emoji
import unicodedata

In [3]:
# Đọc file CSV
df = pd.read_csv("/kaggle/input/data-extracted/dataset_extracted.csv")

print("=== Head ===")
print(df.head())

print("\n=== Shape ===")
print(df.shape)

print("\n=== Info ===")
print(df.info())

=== Head ===
                                             summary  \
0  1. Nội dung sơ lược: Bài viết chỉ trích Phạm V...   
1  1. Nội dung sơ lược: Bài viết chỉ trích Phạm V...   
2  1. Nội dung sơ lược: Bài viết chỉ trích Phạm V...   
3  1. Nội dung sơ lược: Bài viết chỉ trích Phạm V...   
4  1. Nội dung sơ lược: Bài viết chỉ trích Phạm V...   

                                             comment            label  
0  luận điệu của bọn phản động, sỏ lá, 3/// viết ...  KHONG_PHAN_DONG  
1  vậy ông bảo đại, ông diệm, ông thiệu là đảng v...  KHONG_PHAN_DONG  
2                              muôn đời của đám 3///  KHONG_PHAN_DONG  
3  già rồi mà đần vậy cháu ? cộng sản đánh mỹ, đá...  KHONG_PHAN_DONG  
4  đúng là 3/// xỏ lá, bác hồ mất nên các bác khó...  KHONG_PHAN_DONG  

=== Shape ===
(11827, 3)

=== Info ===
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11827 entries, 0 to 11826
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ---

In [4]:
selected_df = df[['summary', 'comment', 'label']]
print(selected_df.head())

print("\n=== Shape ===")
print(selected_df.shape)

print("\n=== Info ===")
print(selected_df.info())

                                             summary  \
0  1. Nội dung sơ lược: Bài viết chỉ trích Phạm V...   
1  1. Nội dung sơ lược: Bài viết chỉ trích Phạm V...   
2  1. Nội dung sơ lược: Bài viết chỉ trích Phạm V...   
3  1. Nội dung sơ lược: Bài viết chỉ trích Phạm V...   
4  1. Nội dung sơ lược: Bài viết chỉ trích Phạm V...   

                                             comment            label  
0  luận điệu của bọn phản động, sỏ lá, 3/// viết ...  KHONG_PHAN_DONG  
1  vậy ông bảo đại, ông diệm, ông thiệu là đảng v...  KHONG_PHAN_DONG  
2                              muôn đời của đám 3///  KHONG_PHAN_DONG  
3  già rồi mà đần vậy cháu ? cộng sản đánh mỹ, đá...  KHONG_PHAN_DONG  
4  đúng là 3/// xỏ lá, bác hồ mất nên các bác khó...  KHONG_PHAN_DONG  

=== Shape ===
(11827, 3)

=== Info ===
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11827 entries, 0 to 11826
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   summ

## 1.Normalize Unicode (NFC) + lowercase

In [5]:
import unicodedata
def normalize_unicode_lower(text):
    text = unicodedata.normalize('NFC', text)
    return text.lower()

## 2.Remove Emoji, links/HTML/mentions/hashtags/UI indicators

In [6]:
import re
import emoji
import unicodedata

# Cải thiện các pattern emoticon từ minimal_cleaning.py
EMOTICON_PATTERNS = [
    r":\)+",       # :), :)), :))), ...
    r":\(+",       # :(, :((, ...
    r":v+",        # :v, :vvv, ...
    r":V+",        # :V, :VV, ... (thêm từ minimal_cleaning.py)
    r"=+\)+",      # =), =)), ...
    r"=+\(+",      # =(, =((, ...
    r":d+",        # :d, :dd
    r":p+",        # :p, :pp
    r"<3+",        # <3<3<3
    r"=+\]+",      # =], =]], =]]], ...
    r"=+\[+",      # =[, =[[, =[[[ ...
    r":>+",        # :>, :>>, ... (thêm từ minimal_cleaning.py)
    r":<+",        # :<, :<<, ... (thêm từ minimal_cleaning.py)
    r":\(\(",      # :(( (thêm từ minimal_cleaning.py)
    r"=\(\(",      # =(( (thêm từ minimal_cleaning.py)
]

EMOTICON_REGEX = re.compile("|".join(EMOTICON_PATTERNS), re.IGNORECASE)

# Thêm EMOJI_REGEX từ minimal_cleaning.py để trường hợp không có thư viện emoji
EMOJI_REGEX = re.compile(
    "["
    u"\U0001F600-\U0001F64F"  # emoticons
    u"\U0001F300-\U0001F5FF"  # symbols & pictographs
    u"\U0001F680-\U0001F6FF"  # transport & map symbols
    u"\U0001F700-\U0001F77F"  # alchemical symbols
    u"\U0001F780-\U0001F7FF"  # Geometric Shapes
    u"\U0001F800-\U0001F8FF"  # Supplemental Arrows-C
    u"\U0001F900-\U0001F9FF"  # Supplemental Symbols and Pictographs
    u"\U0001FA00-\U0001FA6F"  # Chess Symbols
    u"\U0001FA70-\U0001FAFF"  # Symbols and Pictographs Extended-A
    u"\U00002702-\U000027B0"  # Dingbats
    u"\U000024C2-\U0001F251" 
    "]+", flags=re.UNICODE
)

def remove_emoji_emoticon(text):
    """
    Xóa emoji và emoticon từ văn bản.
    Sử dụng thư viện emoji hoặc fallback sang regex nếu thư viện không hoạt động.
    """
    if not isinstance(text, str):
        return ""
    
    # Xóa emoji bằng thư viện emoji
    try:
        text = emoji.replace_emoji(text, replace=" ")
    except:
        # Fallback nếu thư viện emoji gặp lỗi
        text = EMOJI_REGEX.sub(" ", text)
    
    # Xóa emoticon bằng regex
    text = EMOTICON_REGEX.sub(" ", text)
    
    # Loại bỏ khoảng trắng dư thừa
    text = re.sub(r'\s+', ' ', text).strip()
    
    return text

# Regex
URL_REGEX = re.compile(r'https?://\S+|www\.\S+|\S+\.(com|org|net|co|vn|io)(/\S*)?')
HTML_REGEX = re.compile(r"<[^>]+>")
MENTION_REGEX = re.compile(r"@[\w\._]+")  # Cải thiện để bắt dấu . và _
HASHTAG_REGEX = re.compile(r"#\w+")
UI_INDICATORS = [
    "đã chỉnh sửa", "[đã chỉnh sửa]", "(đã chỉnh sửa)",
    "see more", "xem thêm", "see translation", "xem bản dịch",
    "ẩn bớt", "xem ít hơn", "dịch", "translated", "more", "less",
    "see more reactions"
]

def remove_html_url_mention_hashtag(text):
    """
    Loại bỏ URL, HTML tags, mentions, hashtags và các chỉ báo giao diện.
    """
    if not isinstance(text, str):
        return ""
    
    # Chuẩn hóa Unicode (từ minimal_cleaning.py)
    text = unicodedata.normalize('NFC', text)
    
    # Xóa URL
    text = URL_REGEX.sub(" ", text)
    
    # Xóa HTML tags
    text = HTML_REGEX.sub(" ", text)
    
    # Xóa mentions và hashtags
    text = MENTION_REGEX.sub(" ", text)
    text = HASHTAG_REGEX.sub(" ", text)
    
    # Xóa các chỉ báo giao diện (case insensitive)
    for ind in UI_INDICATORS:
        text = re.sub(r'(?i)' + re.escape(ind), " ", text)
    
    # Loại bỏ dấu câu riêng lẻ
    text = re.sub(r'(?<!\w)[\^\'\`\~\"\,\.]+(?!\w)', ' ', text)
    
    # Làm sạch khoảng trắng dư thừa
    text = re.sub(r'\s+', ' ', text).strip()
    
    return text

## 3.Remove punctuation + strip extra spaces

In [7]:
ALLOWED_SPECIAL_CHARS = r"+/\|=&\.'|"
VIET_CHARACTERS = (
    r"àáạảãâầấậẩẫăằắặẳẵ"
    r"èéẹẻẽêềếệểễ"
    r"ìíịỉĩ"
    r"òóọỏõôồốộổỗơờớợởỡ"
    r"ùúụủũưừứựửữ"
    r"ỳýỵỷỹ"
    r"đ"
)
INVALID_CHAR_REGEX = regex.compile(
    f"[^{VIET_CHARACTERS}0-9a-zA-Z {ALLOWED_SPECIAL_CHARS}]+",
    flags=regex.IGNORECASE
)

def remove_punctuation_keep_vietnamese(text):
    if not isinstance(text, str):
        return ""
    return INVALID_CHAR_REGEX.sub(" ", text)

def strip_extra_spaces(text):
    if not isinstance(text, str):
        return ""
    return regex.sub(r"\s+", " ", text).strip()

In [8]:
def remove_punctuation_completely(text):
    """Loại bỏ hoàn toàn dấu câu, chỉ giữ chữ, số và space"""
    if not isinstance(text, str):
        return ""
    # Chỉ giữ lại chữ Việt, số, chữ Latin và khoảng trắng
    pattern = f"[^{VIET_CHARACTERS}0-9a-zA-Z\\s]+"
    text = re.sub(pattern, " ", text)
    return re.sub(r'\s+', ' ', text).strip()

## 4.Reduce elongated characters

In [9]:
import regex

def reduce_elongated(text):
    if not isinstance(text, str):
        return ""
    # \p{L} khớp mọi ký tự chữ Unicode (bao gồm cả tiếng Việt có dấu)
    pattern = regex.compile(r"([\p{L}])\1{2,}", flags=regex.IGNORECASE)
    return pattern.sub(r"\1\1", text)

## 5. De-teencode, Decode Abbreviate 

In [10]:
import re
import uuid
import json

# Load dictionary với error handling
try:
    with open('/kaggle/input/d/taitran501/dictionary/text_normalization_map.json', encoding='utf-8') as f:
        norm_dict = json.load(f)
    print(f"✅ Đã load dictionary với {len(norm_dict)} entries")
except FileNotFoundError:
    print("❌ Không tìm thấy file dictionary")
    norm_dict = {}
except Exception as e:
    print(f"❌ Lỗi khi load dictionary: {str(e)}")
    norm_dict = {}

def normalize_teencode_improved(text, dictionary):
    """
    Cải thiện thuật toán normalize teencode với pattern matching tốt hơn
    """
    if not isinstance(text, str):
        return ""
    
    # Phân loại entries thành 3 nhóm để xử lý riêng biệt
    pure_symbol_entries = []     # Chỉ ký tự đặc biệt: "///", "+", "="
    mixed_entries = []           # Có chữ/số + ký tự đặc biệt: "3///", "v+", "việt+"
    word_entries = []            # Chỉ chữ/số: "ba", "cs", "csvn"
    
    for slang, standard in dictionary.items():
        if re.fullmatch(r"[^\w\s]+", slang):
            # Chỉ toàn ký tự đặc biệt
            pure_symbol_entries.append((slang, standard))
        elif re.search(r"[^\w\s]", slang):
            # Có ký tự đặc biệt mixed với chữ/số
            mixed_entries.append((slang, standard))
        else:
            # Chỉ chữ/số/space
            word_entries.append((slang, standard))
    
    # Sắp xếp mỗi nhóm theo độ dài giảm dần
    pure_symbol_entries = sorted(pure_symbol_entries, key=lambda x: -len(x[0]))
    mixed_entries = sorted(mixed_entries, key=lambda x: -len(x[0]))
    word_entries = sorted(word_entries, key=lambda x: -len(x[0]))
    
    # Xử lý theo thứ tự: Mixed → Pure Symbol → Word
    processing_order = mixed_entries + pure_symbol_entries + word_entries
    
    temp_map = {}
    
    for slang, standard in processing_order:
        # Tạo token tạm unique
        temp_token = f"__TMP_{uuid.uuid4().hex[:8]}__"
        
        # Chọn pattern phù hợp cho từng loại entry
        if slang in [entry[0] for entry in pure_symbol_entries]:
            # Pure symbols: exact match, không cần word boundary
            pattern = re.compile(re.escape(slang), flags=re.IGNORECASE)
            
        elif slang in [entry[0] for entry in mixed_entries]:
            # Mixed entries: cần xử lý cẩn thận word boundary
            if slang.startswith(tuple('abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789')):
                # Bắt đầu bằng chữ/số: cần word boundary ở đầu
                if slang.endswith(tuple('abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789')):
                    # Kết thúc bằng chữ/số: cần word boundary ở cuối
                    pattern = re.compile(rf"\b{re.escape(slang)}\b", flags=re.IGNORECASE)
                else:
                    # Kết thúc bằng ký tự đặc biệt: chỉ cần word boundary ở đầu
                    pattern = re.compile(rf"\b{re.escape(slang)}(?!\w)", flags=re.IGNORECASE)
            else:
                # Bắt đầu bằng ký tự đặc biệt
                if slang.endswith(tuple('abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789')):
                    # Kết thúc bằng chữ/số: cần word boundary ở cuối
                    pattern = re.compile(rf"(?<!\w){re.escape(slang)}\b", flags=re.IGNORECASE)
                else:
                    # Kết thúc bằng ký tự đặc biệt: không cần word boundary
                    pattern = re.compile(re.escape(slang), flags=re.IGNORECASE)
                    
        else:
            # Word entries: dùng word boundary thông thường
            pattern = re.compile(rf"\b{re.escape(slang)}\b", flags=re.IGNORECASE)
        
        # Thực hiện replace
        new_text, count = pattern.subn(temp_token, text)
        if count > 0:
            temp_map[temp_token] = standard
            text = new_text  # Cập nhật text ngay để tránh conflict
    
    # Cuối cùng, thay token tạm bằng từ chuẩn
    for token, standard in temp_map.items():
        text = text.replace(token, standard)
    
    return text

# Function để apply lên dataframe
def apply_improved_normalize(text):
    """Wrapper function để apply lên pandas DataFrame"""
    return normalize_teencode_improved(text, norm_dict)

✅ Đã load dictionary với 462 entries


## 6. Deduplication 

In [11]:
def deduplicate_comments(df, col):
    before = len(df)
    df_nodup = df.drop_duplicates(subset=[col]).reset_index(drop=True)
    after = len(df_nodup)
    return df_nodup

In [12]:
# ===== MAIN PREPROCESSING PIPELINE =====
selected_df = selected_df.copy()

# Store original count
original_count = len(selected_df)
print(f"Starting with {original_count:,} comments")

# 1. Normalize Unicode (NFC) + lowercase
print("1️. Normalizing Unicode and converting to lowercase")
selected_df['comment_clean'] = selected_df['comment'].apply(normalize_unicode_lower)

# 2. Loại bỏ emoji, HTML, URL, mentions, hashtags, UI indicators
print("2️. Removing emoji, HTML, URLs, mentions, hashtags")
selected_df['comment_clean'] = selected_df['comment_clean'].apply(remove_html_url_mention_hashtag)
selected_df['comment_clean'] = selected_df['comment_clean'].apply(remove_emoji_emoticon)

# 3. Xóa ký tự không phải tiếng Việt/tiếng Anh + làm sạch khoảng trắng
print("3️. Removing non-Vietnamese/English characters")
selected_df['comment_clean'] = selected_df['comment_clean'].apply(remove_punctuation_keep_vietnamese)
selected_df['comment_clean'] = selected_df['comment_clean'].apply(strip_extra_spaces)

# 4. Giảm lặp ký tự kéo dài
print("4️. Reducing elongated characters")
selected_df['comment_clean'] = selected_df['comment_clean'].apply(reduce_elongated)

# 5. De-teencode với thuật toán cải thiện
print("5️. Applying improved de-teencode normalization")
selected_df['comment_clean'] = selected_df['comment_clean'].apply(apply_improved_normalize)

# 6. Loại bỏ hoàn toàn dấu câu
print("6️. Removing all punctuation")
selected_df['comment_clean'] = selected_df['comment_clean'].apply(remove_punctuation_completely)

# 7. Làm sạch khoảng trắng cuối cùng
print("7️.Final space cleanup")
selected_df['comment_clean'] = selected_df['comment_clean'].apply(strip_extra_spaces)

# 8. Loại bỏ các comment trống hoặc quá ngắn
print("8️.Filtering out empty or too short comments")
before_filter = len(selected_df)
selected_df = selected_df[
    (selected_df['comment_clean'].str.len() >= 2) & 
    (selected_df['comment_clean'].str.strip() != "") &
    (selected_df['comment_clean'] != "")
].reset_index(drop=True)
after_filter = len(selected_df)
print(f"  → Removed {before_filter - after_filter:,} short/empty comments")

# 9. Loại bỏ dòng trùng lặp
before_dedup = len(selected_df)
selected_df = deduplicate_comments(selected_df, col='comment_clean')
after_dedup = len(selected_df)
print(f"  → Removed {before_dedup - after_dedup:,} duplicate comments")

# ===== STATISTICS =====
print("\n" + "="*40)
print("="*40)
print(f"  • Original comments:      {original_count:,}")
print(f"  • After filtering:        {after_filter:,} ({after_filter/original_count*100:.1f}%)")
print(f"  • Final comments:         {after_dedup:,} ({after_dedup/original_count*100:.1f}%)")
print(f"  • Total reduction:        {original_count - after_dedup:,} comments ({(original_count - after_dedup)/original_count*100:.1f}%)")

# Label distribution
print(f"\n Label Distribution:")
label_counts = selected_df['label'].value_counts()
for label, count in label_counts.items():
    percentage = count / len(selected_df) * 100
    print(f"  • {label:<20}: {count:,} ({percentage:.1f}%)")

# ===== RESULTS =====
print(f"\n results:")
print("-" * 30)

# Create final export dataframe
export_df = selected_df[['summary', 'comment_clean', 'label']].copy()

# Export to CSV
export_df.to_csv("/kaggle/working/final_preprocessed_output.csv", index=False, encoding="utf-8")
print(f" Saved {len(export_df):,} preprocessed comments to: final_preprocessed_output.csv")

print(f" Final dataset: {len(export_df):,} clean comments")

Starting with 11,827 comments
1️. Normalizing Unicode and converting to lowercase
2️. Removing emoji, HTML, URLs, mentions, hashtags
3️. Removing non-Vietnamese/English characters
4️. Reducing elongated characters
5️. Applying improved de-teencode normalization
6️. Removing all punctuation
7️.Final space cleanup
8️.Filtering out empty or too short comments
  → Removed 0 short/empty comments
  → Removed 141 duplicate comments

  • Original comments:      11,827
  • After filtering:        11,827 (100.0%)
  • Final comments:         11,686 (98.8%)
  • Total reduction:        141 comments (1.2%)

 Label Distribution:
  • KHONG_LIEN_QUAN     : 6,414 (54.9%)
  • KHONG_PHAN_DONG     : 4,001 (34.2%)
  • PHAN_DONG           : 1,271 (10.9%)

 results:
------------------------------
 Saved 11,686 preprocessed comments to: final_preprocessed_output.csv
 Final dataset: 11,686 clean comments
